In [37]:
from bs4 import BeautifulSoup
import csv
import re
import unicodedata
import hashlib
from difflib import SequenceMatcher 

In [ ]:

AR_DIACRITICS = re.compile(r'[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]')

In [39]:
URL_RE = re.compile(r'https?://\S+')

def strip_urls(s: str) -> str:
    return URL_RE.sub('', s).strip()

In [ ]:
import re

def is_rules_announcement(text: str) -> bool:
    """يرجع True إذا كانت الرسالة إعلان/قوانين/ترحيب عام، فنستثنيها."""
    if not text:
        return False

    t  = strip_urls(text)
    tn = normalize_ar(t)

   
    if "قوانين المجموعه" in tn or "قوانين الغروب" in tn or "قوانين الجروب" in tn:
        return True
    if "سيتم حذفك" in tn or "سيتم حذفكم" in tn:
        return True

  
    dash_count = tn.count("-") + tn.count("•") + tn.count("–")
    no_count   = tn.count("عدم")

    if dash_count >= 5:
        return True
    if dash_count >= 3 and no_count >= 2:
        return True

    
    if any(e in t for e in ("🔈","📢","📣","📌","⚠️")) and ("قوانين" in tn or no_count >= 2):
        return True

   
    if ("اهلا" in tn or "مرحبا" in tn or "ترحيب" in tn) and ("قوانين" in tn or "الالتزام" in tn):
        return True


    line_count = len([x for x in re.split(r"\s*\n+\s*", t) if x.strip()])
    if line_count >= 6 and ("قوانين" in tn or no_count >= 2 or dash_count >= 3):
        return True

    return False


In [ ]:
MENTION_RE = re.compile(r'@[\w\d_]+')
MULTISPACE_RE = re.compile(r'\s+')

def normalize_answer_text(s: str) -> str:
    """تطبيع نص الجواب قبل المقارنة/الهاش."""
    s = strip_urls(s or '')
    s = MENTION_RE.sub('', s)
    s = normalize_ar(s)           
    s = s.lower()
    s = MULTISPACE_RE.sub(' ', s).strip()
    return s

def text_hash(s: str) -> str:
    return hashlib.sha1(s.encode('utf-8')).hexdigest()

class TextDeduper:
    def __init__(self, sim_threshold: float = 0.90, approx_window: int = 200):
        self.sim_threshold = sim_threshold
        self.approx_window = approx_window
        self._seen_hashes = set()
        self._recent_norm_texts = []  

    def is_duplicate(self, text: str):
        """يرجع (is_dup: bool, matched_norm: str | None)"""
        n = normalize_answer_text(text)
        if not n:
            return False, None  
        h = text_hash(n)
        if h in self._seen_hashes:
            return True, n 

        
        for cand in reversed(self._recent_norm_texts[-self.approx_window:]):
            if len(cand) < 12 and len(n) < 12:
                
                ratio = SequenceMatcher(a=cand, b=n).ratio()
                if ratio >= max(self.sim_threshold, 0.95):
                    return True, cand
            else:
                ratio = SequenceMatcher(a=cand, b=n).ratio()
                if ratio >= self.sim_threshold:
                    return True, cand

        return False, None

    def add(self, text: str):
        """يحاول إضافة النص. يرجع (added: bool, norm_text: str)."""
        n = normalize_answer_text(text)
        if not n:
            return False, n
        h = text_hash(n)
        if h in self._seen_hashes:
            return False, n
      
        is_dup, _ = self.is_duplicate(text)
        if is_dup:
            return False, n
      
        self._seen_hashes.add(h)
        self._recent_norm_texts.append(n)
        return True, n

In [ ]:
global_deduper = TextDeduper(sim_threshold=0.90, approx_window=200)

def collect_answers_for_question(messages_after_q):
    local_deduper  = TextDeduper(sim_threshold=0.90, approx_window=100)
    unique_answers = []

    for msg in messages_after_q:
        ans_text = msg.get("text")  
  

    
        added_local, _ = local_deduper.add(ans_text)
        if not added_local:
            continue

  
        added_global, _ = global_deduper.add(ans_text)
        if not added_global:
            continue

        unique_answers.append(ans_text)

    return unique_answers


In [ ]:
def normalize_ar(s: str) -> str:
    s = (s or '').strip()

    s = s.replace("أ","ا").replace("إ","ا").replace("آ","ا")

    s = s.replace("ى","ي").replace("ة","ه")

    s = AR_DIACRITICS.sub("", s)

    s = s.replace("ـ", "")

    s = re.sub(r"[^\w\s]", " ", s)

    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [ ]:
def clean_text(t: str) -> str:
    if t is None:
        return ""
    t = " ".join(t.split())

    return t

In [45]:


def is_meaningful(text):
    if not text:
        return False
    if len(text) < 4:
        return False
    return True


In [ ]:
def sender_bonus(sender: str) -> int:
    s = (sender or "").strip().lower()
   
    return 10 if s == "saher" else 0

def compute_qscore(msg: dict) -> int:
    base = question_score(msg.get("text", ""))
    return min(100, base + sender_bonus(msg.get("sender")))


In [ ]:
# قوائم الإشارات
AR_Q_START = ("كيف","شو يعني","ماذا","ليش","لماذا","هل","متى","أين","كم","شلون","ايش","شنو","فليش","أيمت","سؤال","سوال")

AR_Q_PHRASES = ("شو رايكم","ايه رايكم","ما رايكم","شو رايك","ايه رايك","بفضل","ركز","كوتش","يفضل","نبهك")

IMP_WORDS = ("امشي","ما تعاكس","ما تعاكسوا","أهم","أفضل","خطأ","ممنوع","وحش","نصيحتي"
             ,"انصحك","نصيحة","برافو","احسبو","احسبوا","مهم","مهمة","الطمع","استنو","استنوا"
             ,"بتلاحظو","بتلاحظو","درس","غلطي","ستنو","بتعوض","تحذير","راحة","نفسية","القرارات","بتاثر"
             ,"خطة","احفظي","الدرس","ندمان","علمني","أمورك تمام","لاتمشي","خلي","مستواي","ضد","التزم","القاعدة","قاعدة","شفتو","نوه","ادخل","حابب","تريدنغ")

DOMAIN_WORD = ("راسمالك","رأسمال","ذيل","قمة","مقاومة","دعم","فريم","تشارت","شارت",
               "تحليل","سوينغ","صفقة","سكالبينغ","الفريم","الترند","ترند","رأس مال","ذهب","دهب","ستوب","هدف","التداول","صفقات","بيع","العامل النفسي")


AR_Q_START_N   = set(normalize_ar(w) for w in AR_Q_START)
AR_Q_PHRASES_N = tuple(normalize_ar(p) for p in AR_Q_PHRASES)
IMP_WORDS_N    = set(normalize_ar(w) for w in IMP_WORDS)
DOMAIN_WORD_N  = tuple(normalize_ar(w) for w in DOMAIN_WORD)

URL_RE = re.compile(r'https?://\S+')
TOKEN_SPLIT_RE = re.compile(r"[^\w\u0600-\u06FF]+", flags=re.UNICODE)  # فاصلات/مسافات/رموز

def strip_urls(s: str) -> str:
    return URL_RE.sub('', s or '').strip()



def split_tokens(tn: str):
    toks = tn.split()
    return toks

def question_score(text: str) -> int:
    if not text:
        return 0

    t  = strip_urls(text)
    tn = normalize_ar(t)


    if is_rules_announcement(text):
        return 0

    score = 0


    if "?" in t or "؟" in t:
        score += 10


    tokens    = split_tokens(tn)
    token_set = set(tokens)


    if token_set.intersection(AR_Q_START_N):
        score += 15


    if any(phrase in tn for phrase in AR_Q_PHRASES_N):
        score += 15


    if token_set.intersection(IMP_WORDS_N):
        score += 40


    dom_unique = set()
    for w in DOMAIN_WORD_N:
        if " " in w:
            if w in tn:
                dom_unique.add(w)
        else:
            if w in token_set:
                dom_unique.add(w)

    if len(dom_unique) >= 2:
        score += 40
    elif len(dom_unique) == 1:
        score += 20


    letters_only = re.sub(r"\W+", "", tn, flags=re.UNICODE)
    if len(letters_only) < 8:
        score -= 20
    if len(letters_only) == 0 and len(tn) > 0:
        score -= 20

    return max(0, min(100, score))

In [48]:
def looks_like_question(text: str, threshold: int = 40) -> bool:
    return question_score(text) >= threshold

In [ ]:
def parse_messages_simple(html_file):
    with open(html_file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    all_blocks = soup.find_all("div", class_=True)

    messages = []
    current_date = None
    last_sender = None

    for blk in all_blocks:
        classes = blk.get("class", [])

        if not (("message" in classes) or ("pull_right" in classes)):
            continue


        if "date" in classes:
            #body = blk.find("div", class_="body") or blk.find("div", class_=lambda c: c and "body" in c)
            body = blk.get("title", "")

            if body:
                txt = clean_text(body)
                if txt:
                    current_date = txt[0:10]
                    #print(current_date[0:10])
            continue


        body = blk.find("div", class_="body") or blk.find("div", class_=lambda c: c and "body" in c)
        if not body:
            continue

        from_name_el = body.find("div", class_="from_name") or body.find("div", class_=lambda c: c and "from_name" in c)
        if from_name_el:
            sender = clean_text(from_name_el.get_text(" ", strip=True))
            last_sender = sender
        else:
            sender = last_sender or "unknown"

        text_el = body.find("div", class_="text") or body.find("div", class_=lambda c: c and "text" in c)
        text = clean_text(text_el.get_text(" ", strip=True)) if text_el else ""

        if not text:
            continue

        tg_id = blk.get("id", "")

        messages.append({
            "tg_id": tg_id,
            "date": current_date,
            "sender": sender,
            "text": text,
            "   " : "   "
        })

        if len(messages) >= 10000 : break

    return messages

In [ ]:

global_deduper = TextDeduper(sim_threshold=0.90, approx_window=200)
global_answer_owner = {}  

def build_candidates_simple(
    messages,
    window_after: int = 5,
    threshold: int = 40,
    max_answers: int = 3,
    min_answer_chars: int = 8,
    require_answers: bool = True,
    min_qscore_include_if_no_answers: int = 70, 
):
    rows = []
    seen_questions = set()

    for i, m in enumerate(messages):
      
        if is_rules_announcement(m.get("text", "")):
            continue

        qscore = compute_qscore(m)
        if qscore < threshold:
            continue

        q_id = m.get("tg_id") or f"i{i}"
        if q_id in seen_questions:
            continue
        seen_questions.add(q_id)

        q = m
        candidates = []
        local_deduper = TextDeduper(sim_threshold=0.90, approx_window=100)

        j = i + 1
        end = min(len(messages), i + 1 + window_after)
        while j < end:
            reply = messages[j]

         
            if is_rules_announcement(reply.get("text", "")):
                j += 1
                continue

            if compute_qscore(reply) >= threshold:
                break

            if reply.get("sender") != q.get("sender") and is_meaningful(reply.get("text", "")):
                raw = (reply.get("text") or "").strip()
                n = normalize_answer_text(raw)
                if len(n) >= min_answer_chars:
                    added_local, _ = local_deduper.add(raw)
                    if not added_local:
                        j += 1; continue
                    added_global, ng = global_deduper.add(raw)
                    if not added_global:
                        j += 1; continue
                    global_answer_owner.setdefault(ng, q_id)
                    candidates.append(raw)
                    if max_answers and len(candidates) >= max_answers:
                        break
            j += 1

  
        include_row = (
            len(candidates) > 0
            or not require_answers
            or qscore >= min_qscore_include_if_no_answers
        )

        if include_row:
            rows.append({
                "tg_msg_id": q_id,
                "date": q.get("date") or "",
                "sender": q.get("sender") or "",
                "question_text": q.get("text") or "",
                "answer_candidates": " ||| ".join(candidates) if candidates else "",
                "q_score": qscore,
            })

    return rows


In [51]:
def main():
    html_in = "messages22.html"
    csv_out = "candidates_simple22.csv"

    msgs = parse_messages_simple(html_in)
    rows = build_candidates_simple(msgs, window_after=5)

    with open(csv_out, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["tg_msg_id", "date", "sender", "question_text", "answer_candidates","q_score"])
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

    print("Parsed messages:", len(msgs))
    print("Candidate questions found:", len(rows))
    print("Output CSV:", csv_out)


In [52]:
main()

Parsed messages: 82
Candidate questions found: 2
Output CSV: candidates_simple22.csv
